In [1]:
import pandas as pd
import requests
import os
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [2]:
for dir_name in ['data', 'data/gameweek_scores', 'data/match_stats', 'data/player_stats']:
    os.makedirs(dir_name, exist_ok=True)

base_url = "https://api.sofascore.com/api/v1/unique-tournament/17/season/41886/"
current_gw = 5

### Team

##### Overall

In [3]:
r = requests.get(base_url + "standings/total").json()
stats = r['standings'][0]['rows']

team_stats_overall = pd.DataFrame([{
    'name': team['team']['name'],
    'position': team['position'],
    'matches': team['matches'],
    'wins': team['wins'],
    'losses': team['losses'],
    'draws': team['draws'],
    'scoresFor': team['scoresFor'],
    'scoresAgainst': team['scoresAgainst'],
    'points': team['points']
} for team in stats])

##### Home

In [4]:
r = requests.get(base_url + "standings/home").json()
stats = r['standings'][0]['rows']

team_stats_home = pd.DataFrame([{
    'name': team['team']['name'],
    'position': team['position'],
    'matches': team['matches'],
    'wins': team['wins'],
    'losses': team['losses'],
    'draws': team['draws'],
    'scoresFor': team['scoresFor'],
    'scoresAgainst': team['scoresAgainst'],
    'points': team['points']
} for team in stats])

##### Away

In [5]:
r = requests.get(base_url + "standings/away").json()
stats = r['standings'][0]['rows']

team_stats_away = pd.DataFrame([{
    'name': team['team']['name'],
    'position': team['position'],
    'matches': team['matches'],
    'wins': team['wins'],
    'losses': team['losses'],
    'draws': team['draws'],
    'scoresFor': team['scoresFor'],
    'scoresAgainst': team['scoresAgainst'],
    'points': team['points']

} for team in stats])

### Gameweek

In [6]:
def get_gameweek_matches(gw_no):
    try:
        gw_score = pd.read_json('data/gameweek_scores/scores_gw{}.json'.format(gw_no))
    except ValueError:
        r = requests.get(base_url + "events/round/{}".format(gw_no)).json()
        gw_score = pd.DataFrame([{
            'gameWeek': gw_no,
            'matchId': event['id'],
            'homeTeam': event['homeTeam']['name'],
            'homeScore': event['homeScore']['current'],
            'awayScore': event['awayScore']['current'],
            'awayTeam': event['awayTeam']['name'],
        } for event in r['events'] if event['status']['code'] == 100])
        with open('gameweek_scores/scores_gw{}.json'.format(gw_no), 'w') as f:
            gw_score.to_json(f, orient='records')
    return gw_score

gw_scores_all = pd.concat([get_gameweek_matches(gw_no) for gw_no in range(1, current_gw+1)])

### Match

In [7]:
def get_match_stats(row):
    match_id = row['matchId']
    try:
        df = pd.read_json('data/match_stats/stats_match{}.json'.format(match_id))
    except ValueError:
        r = requests.get('https://api.sofascore.com/api/v1/event/{}/statistics'.format(match_id)).json()
        stats = []
        for x in r['statistics'][0]['groups']:
            if x['groupName'] not in ['TVData', 'Passes', 'Duels']:
                stats.extend([{
                    'category': x['groupName'],
                    'name': y['name'],
                    'home': y['home'],
                    'away': y['away']
                } for y in x['statisticsItems']])
        df = pd.DataFrame(stats)
        for k, v in row.items():
            df[k] = v
        with open('match_stats/stats_match{}.json'.format(match_id), 'w') as f:
            df.to_json(f, orient='records')
    return df[['gameWeek', 'matchId', 'homeTeam', 'homeScore', 'awayScore', 'awayTeam', 'category', 'name', 'home', 'away']]

team_stats_scores_gw_all = pd.concat([get_match_stats(r) for i, r in gw_scores_all.iterrows()])

### Player

In [8]:
def get_player_stats_in_match(row):
    try:
        df = pd.read_json('data/player_stats/stats_players_match{}.json'.format(row['matchId']))
    except:
        r = requests.get('https://api.sofascore.com/api/v1/event/{}/lineups'.format(row['matchId'])).json()
        player_details = []
        for loc in ['home', 'away']:
            for player in r[loc]['players']:
                d = row.to_dict()
                d.update({
                    'playerId': player['player']['id'],
                    'playerName': player['player']['name'],
                    'teamName': row['{}Team'.format(loc)],
                    'playerPosition': player['player']['position'],
                    'subsititute': player['substitute']
                })
                d.update(player['statistics'])
                player_details.append(d)
        df = pd.DataFrame(player_details)
        with open('player_stats/stats_players_match{}.json'.format(row['matchId']), 'w') as f:
            df.to_json(f, orient='records')
    return df

player_stats_gw_all = pd.concat([get_player_stats_in_match(row) for i, row in gw_scores_all.iterrows()])

### Player Statistics

In [9]:
COLUMNS_G = [
    'gameWeek', 'homeTeam', 'homeScore', 'awayScore', 'awayTeam', 'playerName', 'teamName',
    'playerPosition', 'subsititute', 'minutesPlayed', 'rating', 'savedShotsFromInsideTheBox',
    'saves', 'totalKeeperSweeper', 'accurateKeeperSweeper', 'possessionLostCtrl', 'keyPass',
    'aerialWon', 'aerialLost', 'duelWon', 'duelLost', 'interceptionWon', 'wasFouled', 'fouls',
    'bigChanceCreated', 'goodHighClaim', 'punches', 'goalAssist', 'ownGoals', 'errorLeadToAGoal',
    'errorLeadToAShot', 'penaltySave', 'penaltyConceded'
]
COLUMNS_NG = [
    'gameWeek', 'homeTeam', 'homeScore', 'awayScore', 'awayTeam', 'playerName', 'teamName',
    'playerPosition', 'subsititute', 'minutesPlayed', 'rating', 'goals', 'goalAssist',
    'hitWoodwork', 'shotOffTarget', 'bigChanceMissed', 'onTargetScoringAttempt', 'penaltyWon',
    'penaltyMiss', 'bigChanceCreated', 'keyPass', 'touches', 'accurateCross', 'totalCross',
    'totalOffside', 'wasFouled', 'possessionLostCtrl', 'aerialWon', 'aerialLost', 'duelWon',
    'duelLost', 'dispossessed', 'totalContest', 'wonContest', 'clearanceOffLine',
    'interceptionWon', 'totalTackle', 'lastManTackle', 'fouls', 'blockedScoringAttempt',
    'challengeLost', 'ownGoals', 'errorLeadToAGoal', 'errorLeadToAShot', 'penaltyConceded'
]

In [10]:
def get_player_stats(stats, playerPosition='NG', minutesPlayed=1, gameweeks=[], matchId=None):
    df = stats.fillna(0)
    df = df[df.minutesPlayed >= minutesPlayed]
    df = df[df.playerPosition == playerPosition] if playerPosition != 'NG' else df[df.playerPosition != 'G']
    if gameweeks:
        df = df[df['gameWeek'].isin(gameweeks)]
    if matchId:
        df = df[df.matchId == matchId]
    return df[COLUMNS_G] if playerPosition == 'G' else df[COLUMNS_NG]

In [11]:
players_f = get_player_stats(player_stats_gw_all, 'F')  # Forwards
players_m = get_player_stats(player_stats_gw_all, 'M')  # Mid-fielders
players_d = get_player_stats(player_stats_gw_all, 'D')  # Defenders
players_g = get_player_stats(player_stats_gw_all, 'G')  # Goal Keepers
players_ng = get_player_stats(player_stats_gw_all)  # All NG Players

In [12]:
def stats_filter(data, playerName=None, playerPosition=None, teamName=None, location='All', gameWeeks=[]):
    filtered = data.copy()
    if playerName:
        filtered = filtered[filtered['playerName'] == playerName]
    if playerPosition:
        filtered = filtered[filtered['playerPosition'] == playerPosition]
    if teamName:
        filtered = filtered[filtered['teamName'] == teamName]
    if location == 'Home':
        filtered = filtered[filtered['teamName'] == filtered['homeTeam']]
    elif location == 'Away':
        filtered = filtered[filtered['teamName'] == filtered['awayTeam']]
    if gameWeeks:
        filtered = filtered[filtered['gameWeek'].isin(gameWeeks)]
    return filtered

### All players with respect to Home and/or Away

##### Home and Away

In [13]:
filtered = stats_filter(players_ng, playerPosition='F')
filtered.groupby(['playerName']).agg(sum).sort_values(['rating', 'minutesPlayed'], ascending=[False, False])

,gameWeek,homeScore,awayScore,subsititute,minutesPlayed,rating,goals,goalAssist,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
playerName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Erling Haaland,15,17,7,0,395.0,40.7,9.0,1.0,1.0,9.0,4.0,12.0,1.0,0.0,2.0,2.0,105.0,0.0,0.0,1.0,3.0,24.0,2.0,4.0,9.0,13.0,2.0,5.0,3.0,0.0,0.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0
Aleksandar Mitrović,15,9,6,0,449.0,38.0,5.0,0.0,0.0,9.0,4.0,12.0,1.0,1.0,1.0,5.0,220.0,0.0,0.0,5.0,11.0,88.0,35.0,27.0,61.0,46.0,11.0,12.0,8.0,0.0,1.0,7.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0
Harry Kane,15,8,6,0,450.0,37.4,4.0,0.0,1.0,4.0,1.0,10.0,0.0,1.0,2.0,10.0,214.0,3.0,7.0,1.0,8.0,65.0,6.0,16.0,17.0,40.0,12.0,7.0,2.0,0.0,0.0,1.0,0.0,5.0,5.0,3.0,0.0,0.0,0.0,0.0
Gabriel Jesus,15,8,9,0,434.0,36.9,3.0,3.0,0.0,6.0,3.0,7.0,0.0,0.0,2.0,8.0,256.0,1.0,2.0,4.0,13.0,83.0,15.0,20.0,44.0,58.0,16.0,24.0,13.0,0.0,3.0,3.0,0.0,8.0,5.0,3.0,0.0,0.0,0.0,0.0
Phil Foden,15,17,7,0,368.0,36.7,1.0,2.0,0.0,2.0,2.0,3.0,0.0,0.0,2.0,11.0,291.0,10.0,23.0,0.0,6.0,58.0,3.0,6.0,18.0,20.0,3.0,13.0,4.0,0.0,2.0,5.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
Mason Mount,15,9,5,0,367.0,36.5,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,8.0,273.0,4.0,20.0,0.0,4.0,72.0,1.0,0.0,25.0,19.0,9.0,7.0,4.0,1.0,1.0,16.0,0.0,3.0,5.0,4.0,0.0,0.0,0.0,0.0
Rodrigo,15,9,4,0,359.0,36.0,4.0,1.0,0.0,6.0,2.0,7.0,0.0,0.0,2.0,4.0,164.0,0.0,3.0,0.0,1.0,53.0,9.0,13.0,11.0,31.0,4.0,4.0,0.0,0.0,2.0,1.0,0.0,6.0,6.0,4.0,0.0,0.0,0.0,0.0
Ivan Toney,15,11,6,0,450.0,35.7,2.0,2.0,1.0,7.0,3.0,2.0,0.0,0.0,4.0,10.0,189.0,1.0,6.0,6.0,7.0,66.0,19.0,23.0,31.0,44.0,7.0,8.0,2.0,0.0,4.0,3.0,0.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0
Mohamed Salah,15,16,5,0,450.0,35.7,2.0,2.0,0.0,8.0,3.0,4.0,0.0,0.0,1.0,21.0,242.0,2.0,5.0,7.0,4.0,83.0,2.0,4.0,15.0,37.0,14.0,23.0,8.0,0.0,1.0,1.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0


##### Home

In [14]:
filtered = stats_filter(players_ng, playerPosition='F', location='Home')
filtered.groupby(['playerName']).agg(sum).sort_values(['rating', 'minutesPlayed'], ascending=[False, False])

,gameWeek,homeScore,awayScore,subsititute,minutesPlayed,rating,goals,goalAssist,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
playerName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Erling Haaland,11,14,2,0,227.0,25.9,6.0,1.0,0.0,6.0,0.0,7.0,0.0,0.0,1.0,1.0,40.0,0.0,0.0,1.0,0.0,10.0,2.0,2.0,3.0,8.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
Aleksandar Mitrović,9,7,5,0,269.0,23.5,4.0,0.0,0.0,4.0,4.0,9.0,1.0,0.0,0.0,3.0,143.0,0.0,0.0,5.0,8.0,60.0,24.0,17.0,43.0,29.0,8.0,10.0,7.0,0.0,1.0,4.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
Gabriel Jesus,11,8,4,0,261.0,22.9,3.0,2.0,0.0,6.0,3.0,7.0,0.0,0.0,1.0,6.0,160.0,1.0,2.0,0.0,10.0,52.0,10.0,13.0,25.0,37.0,11.0,12.0,5.0,0.0,1.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,0.0,0.0
Phil Foden,11,14,2,0,190.0,22.2,1.0,2.0,0.0,2.0,2.0,2.0,0.0,0.0,1.0,7.0,147.0,7.0,11.0,0.0,1.0,28.0,1.0,4.0,5.0,12.0,2.0,4.0,0.0,0.0,2.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
Julián Álvarez,11,14,2,2,135.0,22.2,2.0,0.0,1.0,1.0,1.0,3.0,0.0,0.0,1.0,2.0,72.0,0.0,3.0,0.0,2.0,10.0,1.0,3.0,8.0,11.0,1.0,2.0,2.0,0.0,0.0,3.0,0.0,3.0,1.0,4.0,0.0,0.0,0.0,0.0
Wilfried Zaha,9,4,4,0,262.0,21.9,3.0,0.0,0.0,1.0,0.0,4.0,0.0,1.0,1.0,5.0,163.0,1.0,5.0,2.0,13.0,54.0,0.0,0.0,25.0,23.0,11.0,14.0,7.0,0.0,0.0,5.0,0.0,4.0,3.0,1.0,0.0,0.0,0.0,0.0
Riyad Mahrez,11,14,2,1,186.0,21.3,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,4.0,173.0,4.0,12.0,1.0,2.0,22.0,0.0,0.0,6.0,3.0,0.0,3.0,2.0,0.0,1.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
Adam Armstrong,11,4,4,1,192.0,20.8,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,3.0,72.0,1.0,3.0,0.0,2.0,26.0,0.0,3.0,5.0,10.0,2.0,7.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jean-Philippe Mateta,9,4,4,2,106.0,20.8,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,3.0,42.0,0.0,0.0,0.0,0.0,9.0,4.0,2.0,4.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0


##### Away

In [15]:
filtered = stats_filter(players_ng, playerPosition='F', location='Away')
filtered.groupby(['playerName']).agg(sum).sort_values(['rating', 'minutesPlayed'], ascending=[False, False])

,gameWeek,homeScore,awayScore,subsititute,minutesPlayed,rating,goals,goalAssist,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
playerName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Harry Kane,11,3,5,0,270.0,23.3,3.0,0.0,0.0,3.0,1.0,6.0,0.0,1.0,2.0,6.0,140.0,2.0,4.0,1.0,7.0,41.0,4.0,13.0,14.0,26.0,6.0,5.0,2.0,0.0,0.0,1.0,0.0,4.0,4.0,1.0,0.0,0.0,0.0,0.0
Mason Mount,9,5,2,0,232.0,21.8,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,155.0,3.0,13.0,0.0,2.0,43.0,0.0,0.0,10.0,10.0,5.0,4.0,2.0,1.0,0.0,6.0,0.0,2.0,5.0,1.0,0.0,0.0,0.0,0.0
Leandro Trossard,9,3,5,0,254.0,21.5,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,6.0,120.0,4.0,6.0,0.0,3.0,34.0,1.0,3.0,6.0,11.0,3.0,4.0,1.0,0.0,1.0,1.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0
Bryan Mbeumo,9,6,5,0,248.0,21.5,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,4.0,107.0,3.0,11.0,1.0,1.0,36.0,1.0,4.0,8.0,9.0,1.0,5.0,4.0,0.0,2.0,2.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
Ivan Toney,9,6,5,0,270.0,21.3,2.0,0.0,0.0,4.0,2.0,2.0,0.0,0.0,2.0,5.0,124.0,0.0,4.0,6.0,5.0,43.0,14.0,12.0,24.0,26.0,6.0,7.0,2.0,0.0,3.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
Danny Welbeck,9,3,5,1,206.0,21.3,0.0,1.0,0.0,3.0,1.0,1.0,1.0,0.0,2.0,5.0,95.0,0.0,0.0,2.0,2.0,20.0,1.0,7.0,11.0,13.0,1.0,7.0,5.0,0.0,1.0,3.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0
Pedro Neto,9,3,1,0,251.0,21.1,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,9.0,167.0,7.0,23.0,1.0,4.0,46.0,0.0,1.0,13.0,15.0,1.0,13.0,5.0,0.0,2.0,4.0,0.0,2.0,4.0,3.0,0.0,0.0,0.0,0.0
Daniel Podence,9,3,1,1,164.0,20.9,1.0,0.0,0.0,2.0,1.0,2.0,0.0,0.0,1.0,7.0,104.0,1.0,2.0,0.0,4.0,34.0,1.0,0.0,7.0,8.0,3.0,2.0,0.0,0.0,2.0,2.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
Leon Bailey,9,7,2,0,246.0,20.7,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,3.0,131.0,3.0,17.0,1.0,1.0,54.0,3.0,6.0,12.0,22.0,6.0,12.0,6.0,0.0,1.0,2.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0


### Selected player with respect to Home and/or Away

In [209]:
selected_player = 'Erling Haaland'

##### Home and Away

In [216]:
filtered = stats_filter(players_ng, playerName=selected_player)
filtered.sort_values(['playerPosition', 'playerName', 'gameWeek'], ascending=[True, True, False])

,gameWeek,homeTeam,homeScore,awayScore,awayTeam,playerName,teamName,playerPosition,subsititute,minutesPlayed,rating,goals,goalAssist,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
10,4,Manchester City,4,2,Crystal Palace,Erling Haaland,Manchester City,F,False,84.0,9.3,3.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
29,3,Newcastle United,3,3,Manchester City,Erling Haaland,Manchester City,F,False,90.0,7.1,1.0,0.0,1.0,1.0,2.0,3.0,0.0,0.0,1.0,1.0,33.0,0.0,0.0,0.0,2.0,9.0,0.0,2.0,5.0,4.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,2,Manchester City,4,0,AFC Bournemouth,Erling Haaland,Manchester City,F,False,74.0,7.3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,8.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
29,1,West Ham United,0,2,Manchester City,Erling Haaland,Manchester City,F,False,78.0,7.7,2.0,0.0,0.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


##### Home

In [218]:
filtered = stats_filter(players_ng, playerName=selected_player, location='Home')
filtered.sort_values(['playerPosition', 'playerName', 'gameWeek'], ascending=[True, True, False])

,gameWeek,homeTeam,homeScore,awayScore,awayTeam,playerName,teamName,playerPosition,subsititute,minutesPlayed,rating,goals,goalAssist,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
10,4,Manchester City,4,2,Crystal Palace,Erling Haaland,Manchester City,F,False,84.0,9.3,3.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
10,2,Manchester City,4,0,AFC Bournemouth,Erling Haaland,Manchester City,F,False,74.0,7.3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,8.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Away

In [220]:
filtered = stats_filter(players_ng, playerName=selected_player, location='Away')
filtered.sort_values(['playerPosition', 'playerName', 'gameWeek'], ascending=[True, True, False])

,gameWeek,homeTeam,homeScore,awayScore,awayTeam,playerName,teamName,playerPosition,subsititute,minutesPlayed,rating,goals,goalAssist,hitWoodwork,shotOffTarget,bigChanceMissed,onTargetScoringAttempt,penaltyWon,penaltyMiss,bigChanceCreated,keyPass,touches,accurateCross,totalCross,totalOffside,wasFouled,possessionLostCtrl,aerialWon,aerialLost,duelWon,duelLost,dispossessed,totalContest,wonContest,clearanceOffLine,interceptionWon,totalTackle,lastManTackle,fouls,blockedScoringAttempt,challengeLost,ownGoals,errorLeadToAGoal,errorLeadToAShot,penaltyConceded
29,3,Newcastle United,3,3,Manchester City,Erling Haaland,Manchester City,F,False,90.0,7.1,1.0,0.0,1.0,1.0,2.0,3.0,0.0,0.0,1.0,1.0,33.0,0.0,0.0,0.0,2.0,9.0,0.0,2.0,5.0,4.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,1,West Ham United,0,2,Manchester City,Erling Haaland,Manchester City,F,False,78.0,7.7,2.0,0.0,0.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
